<a href="https://colab.research.google.com/github/JavieraTGrey/Galaxy-ages/blob/main/find_best_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ejecutar en colab: https://colab.research.google.com/drive/19hyUoi_ZTp1HYau9g6wqQpUmHiVA7q_t?usp=sharing

In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from ray import train, tune
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from sklearn.model_selection import train_test_split
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.keras import TuneReportCallback
from ray.air.integrations.keras import ReportCheckpointCallback

In [4]:
%load_ext tensorboard

In [5]:
def plot_confusion(true_ages, pred_ages):
  '''
  Plot analogous of confusion matrix for a regression problem
  '''
  plt.scatter(true_ages, pred_ages)
  plt.xlabel('True Age')
  plt.ylabel('Predicted Age (Gyr)')
  plt.title('True vs Predicted Age (Gyr)')
  plt.show()

In [23]:
 def build_model(hp):
  '''
  Build NN model based on hyperparameters
  '''
  model = Sequential()
  model.add(Dense(units=hp['units_1'], activation=hp['activation_1'], input_shape=(X_train.shape[1],))) #TODO: ver input shape. ¿más de un input?
  model.add(Dense(units=hp['units_2'], activation=hp['activation_2']))
  model.add(Dense(1))  # Output layer
  model.compile(optimizer=Adam(hp['lr']), loss=MeanSquaredError())
  return model

In [24]:
def train_model(hp):
  '''
  Train NN model using hyperparameters
  '''
  model = build_model(hp)

  model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, verbose=0,
            callbacks=[TuneReportCallback({"val_loss": "val_loss"})])

In [26]:
# Define search space for hyperparameters
#TODO: editar (esto es solo la arquitectura básica)
search_space = {
    'units_1': tune.choice([64, 128, 256, 512]),
    'units_2': tune.choice([32, 64, 128, 256]),
    'activation_1': tune.choice(['relu', 'tanh']),
    'activation_2': tune.choice(['relu', 'tanh']),
    'lr': tune.loguniform(1e-4, 1e-2)}


In [ ]:
asha_scheduler = ASHAScheduler(
    metric="val_loss",
    mode="min",
    max_t=400,
    grace_period=20,
    reduction_factor=2
)

In [ ]:
tuner = tune.Tuner(
        train_model,
        param_space=search_space,
        tune_config=tune.TuneConfig(
            num_samples=20,
            scheduler=asha_scheduler,
        ),
    )

results = tuner.fit()